# 1. Train the CNN based classifier

## Load the dataset
Don't change this code


In [1]:
# Load and preprocess the data. Don't change this code
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import pickle


# CIFAR10 z-normalization const https://github.com/facebookarchive/fb.resnet.torch/issues/180
cifar10_mean = (0.491, 0.482, 0.447)
cifar10_std = (0.247, 0.244, 0.262)

# Data preprocessing
transform=transforms.Compose([
                              transforms.ToTensor(), # PIL Image to Pytorch tensor
                              transforms.Normalize(cifar10_mean, cifar10_std) # https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transforms%20normalize#torchvision.transforms.Normalize
                              ])

dataset = datasets.CIFAR10("content", train=True, transform = transform ,  download=True)

# Load class names
with open("content/cifar-10-batches-py/batches.meta",'rb') as infile:
  cifar_meta = pickle.load(infile)
labels = cifar_meta['label_names']

# Split dataset into train and val
train_ds, val_ds, _= random_split(dataset, [10000, 2000 ,38000])
batch_size = 256

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size = batch_size)
val_loader = DataLoader(val_ds, batch_size = batch_size)


Files already downloaded and verified


## Function for accuracy checking

Don't change this code

In [2]:
def validate(model,testloader, device = "cpu"):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

  return correct / total

## Implement CNN class for CIFAR10

**In constructor**

Define 2 - 3 convolutional layers

 https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

with corresponding in/out dimensions W_out = 1 + ((W_in - F + 2*P) / S)


Also define max pooling : https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

and fully connected layers: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear


**In forward**

Write code for forward pass.
Remember that first dimension is the batch dimension

In [3]:
import torch.nn as nn
nn.Conv2d?

Init signature:
nn.Conv2d(
    in_channels: int,
    out_channels: int,
    kernel_size: Union[int, Tuple[int, int]],
    stride: Union[int, Tuple[int, int]] = 1,
    padding: Union[str, int, Tuple[int, int]] = 0,
    dilation: Union[int, Tuple[int, int]] = 1,
    groups: int = 1,
    bias: bool = True,
    padding_mode: str = 'zeros',
    device=None,
    dtype=None,
) -> None
Docstring:     
Applies a 2D convolution over an input signal composed of several input
planes.

In the simplest case, the output value of the layer with input size
:math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
can be precisely described as:

.. math::
    \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
    \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


where :math:`\star` is the valid 2D `cross-correlation`_ operator,
:math:`N` is a batch size, :math:`C` denotes a number of channels,
:

In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class TwoLayerCNN(nn.Module):

    def __init__(self, c1_out, c2_out, l1_out, class_nums=10):
        super().__init__()

        # Put your code here ...
        self.conv_seq = nn.Sequential(
            nn.Conv2d(3, c1_out, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(c1_out, c2_out, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(c2_out*8*8, l1_out),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(l1_out, class_nums)
        )

    def forward(self, batch):

        # Put your code here ...
        scores = self.conv_seq(batch)
        return scores

## Train the model




## First Activate tensorboard extension

Use summaryWriter to create logs: https://pytorch.org/docs/stable/tensorboard.html?highlight=summarywriter#torch.utils.tensorboard.writer.SummaryWriter

Display loss and accuracy charts.

You can cange log dir name.

In [5]:
# %load_ext tensorboard
# %tensorboard --logdir runs

### Implement training loop

- Create optimizer,
- Save loss and accuracy values into tensorboard log
- Use GPU to speedup training process.


In [6]:
import torch.optim as optim
import torchvision
import os
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = TwoLayerCNN(c1_out=40, c2_out=200, l1_out=800, class_nums=10)
"""
  Send model to GPU
"""
model.to(device)
model.train()

criterion = nn.CrossEntropyLoss()

"""
  Setup optimizer for your model
"""
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, betas=[0.9, 0.999])

lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.99 ** epoch)

for epoch in range(30):
  model.train()
  for step, (img_batch, labels_batch) in enumerate(train_loader):
    # if step == 2:
    #   break
    output = model(img_batch.to(device)).cpu()
    loss = criterion(output, labels_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  lr_scheduler.step()
  model.eval()
  accuracy = validate(model,val_loader, device)
  """
    Write data to tensorboard logs
  """
  writer.add_scalar('loss', loss.item(), epoch)
  writer.add_scalar('accuracy', accuracy, epoch)

  # You can remove this line when the Tensorboard starts working
  print("Epoch {} Loss {:.9f} Accuracy {:.2f}".format(epoch,loss.item(),accuracy))

writer.flush()


cpu
Epoch 0 Loss 1.567291617 Accuracy 0.41
Epoch 1 Loss 0.952276886 Accuracy 0.50
Epoch 2 Loss 0.561542571 Accuracy 0.53
Epoch 3 Loss 0.365620434 Accuracy 0.56
Epoch 4 Loss 0.234836191 Accuracy 0.56


KeyboardInterrupt: 

### Validat results on test dataset

You must get accuracy above 0.65

In [5]:
test_dataset = datasets.CIFAR10("content",
                           train=False,
                           transform = dataset.transform, # Transforms stay the same
                           download=True)

test_loader = DataLoader(test_dataset, batch_size = batch_size)

accuracy = validate(model,test_loader,device)
print(f"Accuracy on test:{accuracy}")


Files already downloaded and verified
Accuracy on test:0.6352


# 2. Compare different Normalization methods

* Add extra conv layer to your model (3-7)
* Take three different normalization layers: BatchNorm, InstanceNorm, LayerNorm
* Train the model with each of them.
* Plot the loss curve for different normalization in same axis


*Because this task is time consuming it is recommended to perform calculations on a small piece of datastat*

In [ ]:
# Put your code here ...

# Place for brief conclusion:

....


# Ideas for extra work

---
1. Evaluate the impact of the number and size of filters in convolutional layers on the accuracy.

2. Evaluate the impact of the convolutional layers count on the accuracy.

3. Visualize something: filters, activations ...

---







